In [1]:
import jax
import tensorflow as tf
tf.config.set_visible_devices([], device_type='GPU')  # Ensure TF does not see GPU and grab all GPU memory.
tf.random.set_seed(42)  # For reproducibility.

from quantum_transformers.datasets import get_imdb_dataloaders
from quantum_transformers.training import train_and_evaluate
from quantum_transformers.transformers import Transformer

data_dir = '/global/cfs/cdirs/m4392/salcc/data'

2023-08-26 12:38:46.273497: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-08-26 12:38:46.273523: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-08-26 12:38:46.273539: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-26 12:38:47.837952: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Please first ``pip install -U cirq`` to enable related functionality in translation module


In [2]:
for d in jax.devices():
    print(d, d.device_kind)

gpu:0 NVIDIA A100-SXM4-40GB


In [3]:
(imdb_train_dataloader, imdb_valid_dataloader), vocab, tokenizer = get_imdb_dataloaders(batch_size=32, data_dir=data_dir, max_vocab_size=20_000, max_seq_len=512)
print(f"Vocabulary size: {len(vocab)}")
first_batch = next(iter(imdb_train_dataloader))
print(first_batch[0][0])
print(' '.join(map(bytes.decode, tokenizer.detokenize(first_batch[0])[0].numpy().tolist())))

Vocabulary size: 19169
[  141  1693    15   139   161   123   631    34    34     4     4     4
     4     4     4   218    99   220  1473  1848    15   150   241    50
   180    10    61   193   101    10   329   156   645    17   112   478
    10    61   119   414   103   514    97  1254   648    17    31   100
    18    33    31   100    18    33    42   691    97   599   992  4916
 12468   135   230  3431   192    42   691    97   156   472   169    15
   126   230   113    42   156   472  2461   107   187    98   446    96
   605   187    15    96    95  6665 17703   276   124  3424   187   510
   136   158   156   472  5741    17   102   172   803    15    95   315
   243    99   156   472    17    31   100    18    33    31   100    18
    33   112   213   101   138    98   119   167   145   150  2019    96
    95 15091   142    10    61   699   232     4     4    96   123   251
    10    61     4    50   213    15   140    97   158   509   161   256
    15   129   139    98   8

In [4]:
model = Transformer(num_tokens=len(vocab), max_seq_len=512, num_classes=2, hidden_size=64, num_heads=2, num_transformer_blocks=4, mlp_hidden_size=32)
train_and_evaluate(model, imdb_train_dataloader, imdb_valid_dataloader, num_classes=2, num_epochs=30)

Epoch  30/30: 100%|██████████| 781/781 [00:07<00:00, 100.22batch/s, Loss = 1.0221, AUC = 90.36%]

TOTAL TIME = 255.01s
BEST AUC = 94.18% AT EPOCH 2


In [5]:
model = Transformer(num_tokens=len(vocab), max_seq_len=512, num_classes=2, hidden_size=8, num_heads=2, num_transformer_blocks=4, mlp_hidden_size=4)
train_and_evaluate(model, imdb_train_dataloader, imdb_valid_dataloader, num_classes=2, num_epochs=30)

Epoch  30/30: 100%|██████████| 781/781 [00:07<00:00, 98.13batch/s, Loss = 0.7282, AUC = 90.60%] 

TOTAL TIME = 228.20s
BEST AUC = 91.03% AT EPOCH 12
